# Pour le polluant PM10

### On importe les bibliothèques

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Conversion des données en DataFrame

In [3]:

# URL de base
url = "https://data.airpl.org/api/v1/mesure/horaire/"

# Paramètres de base de la requête
base_params = {
    "code_configuration_de_mesure__code_point_de_prelevement__code_polluant": 24,
    "date_heure_tu__range": "2024-1-1,2024-3-31 23:00:00",
    "code_configuration_de_mesure__code_point_de_prelevement__code_station__code_commune__code_departement__in": "44,49,53,72,85",
    "export": "json"
}
# Limite de résultats par requête
limit = 1000
offset = 0

# DataFrame pour stocker tous les résultats
dfPM10 = pd.DataFrame()

while True:
    # Mettre à jour les paramètres avec la limite et l'offset
    params = base_params.copy()
    params.update({
        "limit": limit,
        "offset": offset
    })
    
    # Récupérer les données
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        df_results = pd.DataFrame(data['results'])  # Adapter selon la structure des données JSON
        
        # Ajouter les résultats au DataFrame principal
        dfPM10 = pd.concat([dfPM10, df_results], ignore_index=True)
        
        # Vérifier si le nombre de résultats récupérés est inférieur à la limite
        if len(df_results) < limit:
            break  # Arrêter la boucle si tous les enregistrements ont été récupérés
        
        # Mettre à jour l'offset pour la prochaine itération
        offset += limit
    else:
        print(f"Erreur {response.status_code}: {response.text}")
        break

print(f"Total records retrieved: {len(dfPM10)}")

# Sauvegarder le DataFrame pour utilisation ultérieure
dfPM10.to_pickle('PM10.pkl')

# Afficher les premières lignes du DataFrame
print(dfPM10.head())


Total records retrieved: 41770
                                  id code_polluant code_point_de_prelevement  \
0  2024-03-31 23:00:00FR23003_PM10_1            24              FR23003_PM10   
1  2024-03-31 23:00:00FR23068_PM10_4            24              FR23068_PM10   
2  2024-03-31 23:00:00FR23070_PM10_3            24              FR23070_PM10   
3  2024-03-31 23:00:00FR23078_PM10_2            24              FR23078_PM10   
4  2024-03-31 23:00:00FR23107_PM10_4            24              FR23107_PM10   

  code_station               nom_station               nom_commune  \
0      FR23003              LA MEGRETAIS                    Donges   
1      FR23068                   FROSSAY                   Frossay   
2      FR23070  SAINT ETIENNE DE MONTLUC  Saint-Etienne-De-Montluc   
3      FR23078             SAINT EXUPERY                    Cholet   
4      FR23107            LA CHAUVINIERE                    Nantes   

  code_commune departement_code   departement_nom  code_zone_affich

In [4]:
# Charger le DataFrame depuis le fichier sauvegardé
dfPM10 = pd.read_pickle('PM10.pkl')

# Afficher les types de chaque colonne
print(dfPM10.dtypes)


id                               object
code_polluant                    object
code_point_de_prelevement        object
code_station                     object
nom_station                      object
nom_commune                      object
code_commune                     object
departement_code                 object
departement_nom                  object
code_zone_affichage               int64
date_heure_tu                    object
date_heure_local                 object
valeur_originale                float64
valeur                          float64
validite                         object
code_configuration_de_mesure     object
dtype: object


## Analyse et correction du dataset

### Types et valeurs manquantes par variable

In [5]:
dfPM10.shape

(41770, 16)

In [6]:
dfPM10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41770 entries, 0 to 41769
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            41770 non-null  object 
 1   code_polluant                 41770 non-null  object 
 2   code_point_de_prelevement     41770 non-null  object 
 3   code_station                  41770 non-null  object 
 4   nom_station                   41770 non-null  object 
 5   nom_commune                   41770 non-null  object 
 6   code_commune                  41770 non-null  object 
 7   departement_code              41770 non-null  object 
 8   departement_nom               41770 non-null  object 
 9   code_zone_affichage           41770 non-null  int64  
 10  date_heure_tu                 41770 non-null  object 
 11  date_heure_local              41770 non-null  object 
 12  valeur_originale              40781 non-null  float64
 13  v

In [7]:
dfPM10.isnull().sum()

id                                0
code_polluant                     0
code_point_de_prelevement         0
code_station                      0
nom_station                       0
nom_commune                       0
code_commune                      0
departement_code                  0
departement_nom                   0
code_zone_affichage               0
date_heure_tu                     0
date_heure_local                  0
valeur_originale                989
valeur                          996
validite                          7
code_configuration_de_mesure      0
dtype: int64

Observations : 
- On remarque que pour les colonnes "valeur_originale" et "valeur" et validite on a quelques lignes vides, mais aucune colonne entièrement vide

## Visualiser les différentes données de chaque colonne

In [8]:
for col in dfPM10.select_dtypes('float64'):
    print(f'{col :-<20} {dfPM10[col].unique()}')

valeur_originale---- [ 7.925     4.65      9.25     ... 32.100002 29.349998 51.150002]
valeur-------------- [7.90e+00 4.70e+00 9.30e+00      nan 8.80e+00 8.70e+00 6.70e+00 6.20e+00
 7.30e+00 8.90e+00 8.50e+00 7.40e+00 7.10e+00 1.00e+01 1.50e+01 9.50e+00
 1.40e+01 1.10e+01 7.50e+00 6.60e+00 9.60e+00 6.30e+00 4.80e+00 9.40e+00
 1.20e+01 8.30e+00 7.20e+00 1.30e+01 8.60e+00 9.80e+00 1.90e+00 5.00e+00
 8.00e+00 7.00e+00 5.90e+00 4.30e-01 9.20e+00 5.40e+00 6.90e+00 5.60e+00
 4.40e+00 9.10e+00 5.30e+00 4.30e+00 6.40e+00 6.10e+00 3.50e+00 5.70e+00
 5.10e+00 2.70e+00 3.70e+00 3.60e+00 4.00e+00 4.60e+00 3.00e+00 6.80e+00
 3.40e+00 3.30e+00 3.20e+00 3.80e+00 2.60e+00 3.10e+00 5.50e+00 2.90e+00
 2.50e+00 3.90e+00 2.10e+00 2.80e+00 5.20e+00 8.20e+00 1.70e+00 4.20e+00
 4.10e+00 2.40e+00 1.80e+00 7.80e+00 8.40e+00 2.00e+00 4.50e+00 2.40e+01
 6.00e+00 7.60e+00 8.10e+00 5.80e+00 4.90e+00 7.70e+00 6.50e+00 0.00e+00
 2.20e+00 2.30e+00 9.00e+00 3.80e-01 1.60e+00 9.30e-01 1.30e+00 1.50e+00
 5.30e-01 1.60e+

In [9]:
for col in dfPM10.select_dtypes('int64'):
    print(f'{col :-<20} {dfPM10[col].unique()}')

code_zone_affichage- [ 3  4  1  2  5  6 13  7 11  8]


In [10]:
for col in dfPM10.select_dtypes('object'):
    print(f'{col :-<20} {dfPM10[col].unique()}')

id------------------ ['2024-03-31 23:00:00FR23003_PM10_1' '2024-03-31 23:00:00FR23068_PM10_4'
 '2024-03-31 23:00:00FR23070_PM10_3' ...
 '2024-01-01 00:00:00FR23242_PM10_1' '2024-01-01 00:00:00FR23249_PM10_1'
 '2024-01-01 00:00:00FR23251_PM10_1']
code_polluant------- ['24']
code_point_de_prelevement ['FR23003_PM10' 'FR23068_PM10' 'FR23070_PM10' 'FR23078_PM10'
 'FR23107_PM10' 'FR23110_PM10' 'FR23120_PM10' 'FR23123_PM10'
 'FR23124_PM10' 'FR23152_PM10' 'FR23177_PM10' 'FR23178_PM10'
 'FR23182_PM10' 'FR23188_PM10' 'FR23238_PM10' 'FR23239_PM10'
 'FR23242_PM10' 'FR23249_PM10' 'FR23251_PM10']
code_station-------- ['FR23003' 'FR23068' 'FR23070' 'FR23078' 'FR23107' 'FR23110' 'FR23120'
 'FR23123' 'FR23124' 'FR23152' 'FR23177' 'FR23178' 'FR23182' 'FR23188'
 'FR23238' 'FR23239' 'FR23242' 'FR23249' 'FR23251']
nom_station--------- ['LA MEGRETAIS' 'FROSSAY' 'SAINT ETIENNE DE MONTLUC' 'SAINT EXUPERY'
 'LA CHAUVINIERE' 'LEON BLUM' 'BEAUX ARTS' 'MAZAGRAN' 'LA TARDIERE'
 'DELACROIX' "SAINT DENIS D'ANJOU" '

Observations : 
- on remarque que pour les colonnes de type float64, pour les colonnes "valeur_originale" et "valeur", des valeurs ne sont pas correctes car il y à une présence de "nan" et "-999.".
- Nous allons donc supprimer les lignes ou ces dernières sont présentes, et donc supprimer toutes les valeurs qui seront négatives pour englober.

Certaines colonnes ne semblent pas être du bon types, il faut donc les changer : 
- code_polluant : object --> int64
- code_commune : object --> int64
- departement_code : object --> int64

### Supression des valeurs "nan" et "-999" pour les colonnes "valeur_originale" et "valeur"

In [11]:
# Supprimer les lignes comportant des valeurs "NaN" dans les colonnes "valeur" et "valeur_originale"
dfPM10.dropna(subset=['valeur', 'valeur_originale'], inplace=True)
    
# Supprimer les lignes comportant des valeurs négatives dans les colonnes "valeur" et "valeur_originale"
dfPM10 = dfPM10[(dfPM10['valeur'] >= 0) & (dfPM10['valeur_originale'] >= 0)]

In [12]:
for col in dfPM10.select_dtypes('float64'):
    print(f'{col :-<20} {dfPM10[col].unique()}')

valeur_originale---- [ 7.925     4.65      9.25     ... 32.100002 29.349998 51.150002]
valeur-------------- [7.90e+00 4.70e+00 9.30e+00 8.80e+00 8.70e+00 6.70e+00 6.20e+00 7.30e+00
 8.90e+00 8.50e+00 7.40e+00 7.10e+00 1.00e+01 1.50e+01 9.50e+00 1.40e+01
 1.10e+01 7.50e+00 6.60e+00 9.60e+00 6.30e+00 4.80e+00 9.40e+00 1.20e+01
 8.30e+00 7.20e+00 1.30e+01 8.60e+00 9.80e+00 1.90e+00 5.00e+00 8.00e+00
 7.00e+00 5.90e+00 4.30e-01 9.20e+00 5.40e+00 6.90e+00 5.60e+00 4.40e+00
 9.10e+00 5.30e+00 4.30e+00 6.40e+00 6.10e+00 3.50e+00 5.70e+00 5.10e+00
 2.70e+00 3.70e+00 3.60e+00 4.00e+00 4.60e+00 3.00e+00 6.80e+00 3.40e+00
 3.30e+00 3.20e+00 3.80e+00 2.60e+00 3.10e+00 5.50e+00 2.90e+00 2.50e+00
 3.90e+00 2.10e+00 2.80e+00 5.20e+00 8.20e+00 1.70e+00 4.20e+00 4.10e+00
 2.40e+00 1.80e+00 7.80e+00 8.40e+00 2.00e+00 4.50e+00 2.40e+01 6.00e+00
 7.60e+00 8.10e+00 5.80e+00 4.90e+00 7.70e+00 6.50e+00 0.00e+00 2.20e+00
 2.30e+00 9.00e+00 3.80e-01 1.60e+00 9.30e-01 1.30e+00 1.50e+00 5.30e-01
 1.60e+01 1.80e+

### Changement de type pour les colonnes : 
- code_polluant : object --> int64
- code_commune : object --> int64
- departement_code : object --> int64

In [13]:
# Vérifier si les colonnes existent dans le DataFrame
colonnes_a_convertir = ['code_commune', 'departement_code', 'code_polluant']

# Convertir les colonnes en int64
for col in colonnes_a_convertir:
    if col in dfPM10.columns:
        dfPM10[col] = dfPM10[col].astype('int64')

# Afficher les types de colonnes pour vérifier les conversions
print(dfPM10.dtypes)


id                               object
code_polluant                     int64
code_point_de_prelevement        object
code_station                     object
nom_station                      object
nom_commune                      object
code_commune                      int64
departement_code                  int64
departement_nom                  object
code_zone_affichage               int64
date_heure_tu                    object
date_heure_local                 object
valeur_originale                float64
valeur                          float64
validite                         object
code_configuration_de_mesure     object
dtype: object


In [14]:
from unidecode import unidecode

# Convertir les valeurs de la colonne 'nom_commune' en minuscules et supprimer les accents
dfPM10['nom_commune'] = dfPM10['nom_commune'].apply(lambda x: unidecode(x).lower())

# Afficher les premières lignes pour vérifier les modifications
print(dfPM10.head())


                                  id  code_polluant code_point_de_prelevement  \
0  2024-03-31 23:00:00FR23003_PM10_1             24              FR23003_PM10   
1  2024-03-31 23:00:00FR23068_PM10_4             24              FR23068_PM10   
2  2024-03-31 23:00:00FR23070_PM10_3             24              FR23070_PM10   
4  2024-03-31 23:00:00FR23107_PM10_4             24              FR23107_PM10   
5  2024-03-31 23:00:00FR23110_PM10_3             24              FR23110_PM10   

  code_station               nom_station               nom_commune  \
0      FR23003              LA MEGRETAIS                    donges   
1      FR23068                   FROSSAY                   frossay   
2      FR23070  SAINT ETIENNE DE MONTLUC  saint-etienne-de-montluc   
4      FR23107            LA CHAUVINIERE                    nantes   
5      FR23110                 LEON BLUM             saint-nazaire   

   code_commune  departement_code   departement_nom  code_zone_affichage  \
0         44052 